In [1]:
import os

import numpy as np
import pandas as pd
from pkg_resources import resource_filename

from flip import fisher, utils, data_vector
from flip.covariance import covariance

flip_base = resource_filename("flip", ".")
data_path = os.path.join(flip_base, "data")

/tmp/ipykernel_470034/3270887735.py:5: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_filename
[ 000000.00 ]: 09-20 08:06  root            INFO     No pypower module detected, gridding with this method is unavailable


In [2]:
grid = pd.read_parquet(os.path.join(data_path,"density_data.parquet"))
grid_window = pd.read_parquet(os.path.join(data_path,"grid_window_m.parquet"))

ktt, ptt = np.loadtxt(os.path.join(data_path,"power_spectrum_tt.txt"))
kmt, pmt = np.loadtxt(os.path.join(data_path,"power_spectrum_mt.txt"))
kmm, pmm = np.loadtxt(os.path.join(data_path,"power_spectrum_mm.txt"))

sigmag_fiducial = 3.0

power_spectrum_dict_bias = {"gg": [[kmm, pmm * np.array(grid_window["window_mm"])**2]]}
power_spectrum_dict = {
    "gg": [
        [kmm, pmm * np.array(grid_window["window_mm"])**2],
        [kmt, pmt * np.array(grid_window["window_mt"])],
        [ktt, ptt]
    ]
                      }

In [3]:
DataFisher = data_vector.FisherDens(grid.rename(columns={'density_err': 'density_error', 'rcom': 'rcom_zobs'}).to_dict(orient='list'))
print("Data free parameters: ", DataFisher.free_par)

[ 000000.18 ]: 09-20 08:06  jax._src.xla_bridge INFO     Unable to initialize backend 'cuda': 
[ 000000.18 ]: 09-20 08:06  jax._src.xla_bridge INFO     Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
[ 000000.19 ]: 09-20 08:06  jax._src.xla_bridge INFO     Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory
[ 000000.19 ]: 09-20 08:06  jax._src.xla_bridge WARNING  An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


Data free parameters:  []


In [7]:
size_batch=10_000
number_worker=8
covariance_fit = DataFisher.compute_cov("adamsblake20", power_spectrum_dict, size_batch=size_batch, 
                                          number_worker=number_worker, additional_parameters_values=(sigmag_fiducial,))

print("Covariance free parameters: ", covariance_fit.free_par)

/pscratch/sd/b/bastienc/Soft/.miniconda3/envs/flip/lib/python3.11/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
[ 000025.56 ]: 09-20 08:08  root            INFO     Covariance matrix generated from flip with adamsblake20 model in 1.20e+00 seconds


Covariance free parameters:  ['beta_f', 'bs8']


In [15]:
fisher_properties = {
    "inversion_method": "inverse",
}


In [16]:
parameter_dict = {
    "bs8": 0.8,
    "beta_f": 0.3,
}

Fisher = fisher.FisherMatrix.init_from_covariance(
    covariance_fit,
    DataFisher,
    parameter_dict,
    fisher_properties=fisher_properties,
)

parameter_name_list, fisher_matrix = Fisher.compute_fisher_matrix()

ERR = np.sqrt(np.diag(np.linalg.pinv(fisher_matrix)))

In [17]:
for p, e in zip(parameter_name_list, ERR):
    print(f"Fisher error on {p} = {e:.4f}")

Fisher error on fs8 = 0.0000
Fisher error on bs8 = 0.0322
Fisher error on beta_f = 0.0760
